## Advent of Code 2019

https://adventofcode.com/2019

In [1]:
from pathlib import Path
import numpy as np

In [2]:
input_dir = Path('./inputs')
def load_input(day_num, sep):
    with open(input_dir/f'input{day_num}.txt', 'r') as f:
        data = f.read().split(sep)
    return [dp.strip() for dp in data]

### Day 1

In [4]:
data = load_input(1, '\n')
data = data[:-1]
data = [int(dp) for dp in data]
data[0:2]

[90859, 127415]

#### Part 1

In [5]:
def fuel_fn(mass):
    return int(np.floor(mass/3) - 2)

In [6]:
assert(fuel_fn(12)) == 2
assert(fuel_fn(14)) == 2
assert(fuel_fn(1969)) == 654
assert(fuel_fn(100756)) == 33583

**Answer**

In [7]:
np.sum([fuel_fn(mass) for mass in data])

3262991

#### Part 2

In [8]:
def total_fuel_fn(initial_mass):
    total = 0
    req = fuel_fn(initial_mass)
    total += req
    while req > 0:
        req = max(fuel_fn(req), 0)
        total += req
    return total

In [9]:
assert(total_fuel_fn(14)) == 2
assert(total_fuel_fn(1969)) == 966
assert(total_fuel_fn(100756)) == 50346

**Answer**

In [10]:
np.sum([total_fuel_fn(initial_mass) for initial_mass in data])

4891620

### Day 2

In [11]:
data = load_input(2, ',')
data = data[:-1]
data = [int(dp) for dp in data]
data[0:2]

[1, 0]

#### Part 1

In [12]:
opcode_slice_size = 4

def eval_instruction(inputs, instruction):
    if instruction == 1:
        return np.sum(inputs)
    elif instruction == 2:
        return np.product(inputs)

def eval_opcode_slice(intcode_program, start_index, termination_flag):
    opcode_slice = intcode_program[start_index:start_index+opcode_slice_size]
    instruction = opcode_slice[0]
    if instruction == 99:
        termination_flag = True
        return intcode_program, termination_flag
    inputs = [intcode_program[idx] for idx in opcode_slice[1:3]]
    output_loc = opcode_slice[3]
    eval_result = eval_instruction(inputs, instruction)
    intcode_program[output_loc] = eval_result
    return intcode_program, termination_flag

def eval_opcode(intcode_program):
    termination_flag = False
    start_index = 0
    while termination_flag == False:
        intcode_program, termination_flag = eval_opcode_slice(intcode_program, start_index, termination_flag)
        start_index += opcode_slice_size
    return intcode_program 

In [13]:
prog = [2,4,4,5,99,0]
res = eval_opcode(prog)

In [14]:
data[1:3] = [12, 2]

**Answer**

In [15]:
res = eval_opcode(data.copy())
res[0]

8017076

#### Part 2

In [16]:
def test_inputs(noun, verb):
    prog = data.copy()
    prog[1:3] = [noun, verb]
    res = eval_opcode(prog)
    return res[0]

In [17]:
target = 19690720
endpoint = 100
for noun in range(endpoint):
    for verb in range(endpoint):
        res = test_inputs(noun, verb)
        if res == target:
            print(noun, verb)
            print(100 * noun + verb)
            break

31 46
3146


### Day 3

In [7]:
data = load_input(3, '\n')
data = data[:-1]
len(data)

2

In [11]:
data = [wire.split(',') for wire in data]

#### Part 1

In [307]:
def update_distances(move, coordinates, current_position):
    direction = move[0]
    magnitude = int(move[1:])

    if direction == 'L':
        update = (0, -1)
    elif direction == 'R':
        update = (0, 1)
    elif direction == 'D':
        update = (1, -1)
    elif direction == 'U':
        update = (1, 1)


    wire_spots = [tuple([current_position[j]+(update[1]*i) if update[0] == j else current_position[j] for j in range(2)]) for i in range(magnitude+1)]
    coordinates.extend(wire_spots[1:])
    current_position[update[0]] += magnitude*update[1]
    return coordinates, current_position

def find_intersections(wire_coordinates):
    intersections = list(wire_coordinates[0].intersection(wire_coordinates[1]))
    return intersections

def manhattan(coordinates):
    return np.sum([abs(x) for x in coordinates])

def find_closest_intersection(intersections):
    distances = [manhattan(intersection) for intersection in intersections]
    return (intersections[np.argmin(distances)], np.min(distances))

def eval_wires(wires):
    n_wires = len(wires)
    # Horizontal, Vertical
    current_positions = [[0, 0] for i in range(n_wires)]
    wire_coordinates = [[] for i in range(n_wires)]
          
    for wire_idx, wire in enumerate(wires):
        current_position = current_positions[wire_idx]
        coordinates = wire_coordinates[wire_idx]
        for move in wire:
            coordinates, current_position = update_distances(move, coordinates, current_position)
    
    wire_coordinates_set = [set(coordinate) for coordinate in wire_coordinates]
    intersections = find_intersections(wire_coordinates_set)
    closest_intersection = find_closest_intersection(intersections)
    return wire_coordinates, intersections, closest_intersection

In [308]:
test1 = [
    ['R8','U5','L5','D3'],
    ['U7','R6','D4','L4']
]

test2 = [
    ['R75','D30','R83','U83','L12','D49','R71','U7','L72'],
    ['U62','R66','U55','R34','D71','R55','D58','R83','U400']
]

In [309]:
wire_coordinates, intersections, closest_intersection = eval_wires(test1)
closest_intersection

((3, 3), 6)

In [310]:
wire_coordinates, intersections, closest_intersection = eval_wires(test2)
closest_intersection

((155, 4), 159)

**Answer**

In [311]:
wire_coordinates, intersections, closest_intersection = eval_wires(data)
closest_intersection

((855, 41), 896)

#### Part 2 

In [319]:
def calculate_intersection_steps(wire_coordinates, intersection):
    return np.sum([wire.index(intersection)+1 for wire in wire_coordinates])

def find_min_intersection_steps(wire_coordinates, intersections):
    return np.min([calculate_intersection_steps(wire_coordinates, intersection) for intersection in intersections])

In [320]:
wire_coordinates, intersections, closest_intersection = eval_wires(test1)
find_min_intersection_steps(wire_coordinates, intersections)

30

In [321]:
wire_coordinates, intersections, closest_intersection = eval_wires(test2)
find_min_intersection_steps(wire_coordinates, intersections)

610

**answer**

In [322]:
wire_coordinates, intersections, closest_intersection = eval_wires(data)
find_min_intersection_steps(wire_coordinates, intersections)

16524

### Day 4

#### Part 1

In [2]:
data = range(178416, 676462)

In [3]:
def check_valid(pwd):
    pwd_str = str(pwd)
    has_double = False
    for idx, char in enumerate(pwd_str):
        if idx == 0:
            pass
        else:
            if pwd_str[idx-1] > pwd_str[idx]:
                return False
            if not has_double and pwd_str[idx-1] == pwd_str[idx]:
                has_double = True
    return True if has_double else False

In [4]:
assert check_valid(111111) == True
assert check_valid(223450) == False
assert check_valid(123789) == False

**Answer**

In [5]:
valids = []
for pwd in data:
    if check_valid(pwd):
        valids.append(pwd)
        
len(valids)

1650

#### Part 2

In [13]:
def check_exact_repeat(valid_pwd):
    pwd_str = str(valid_pwd)
    has_exact_double = False
    for idx, char in enumerate(pwd_str):
        if idx is 0:
            pass
        else:
            if not has_exact_double and pwd_str[idx-1] == pwd_str[idx]:
                # look ahead
                if idx == 1:
                    if pwd_str[idx+1] != pwd_str[idx]:
                        has_exact_double = True
                # look behind
                elif idx == 5:
                    if pwd_str[idx-2] != pwd_str[idx]:
                        has_exact_double = True
                # look behind and ahead
                else:
                    if pwd_str[idx+1] != pwd_str[idx] and pwd_str[idx-2] != pwd_str[idx]:
                        has_exact_double = True
                        
    return has_exact_double

In [14]:
assert check_exact_repeat(112233) == True
assert check_exact_repeat(123444) == False
assert check_exact_repeat(111122) == True

**Answer**

In [19]:
valids2 = []
for valid_pwd in valids:
    if check_exact_repeat(valid_pwd):
        valids2.append(valid_pwd)
        
len(valids2)

1129

### Day 5